In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
import numpy as np
%matplotlib inline

In [209]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import roc_curve, auc

In [4]:
from catboost import CatBoostClassifier
from catboost import Pool, cv
import optuna
import pandas_profiling as pp

In [196]:
train = pd.read_csv('data/training_set_features.csv', index_col='respondent_id')

In [197]:
labels = pd.read_csv('data/training_set_labels.csv', index_col='respondent_id')

In [10]:
join_df = train.join(labels)

In [11]:
join_df

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation,h1n1_vaccine,seasonal_vaccine
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN,0,0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe,0,1
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,Own,Not in Labor Force,qufhixun,Non-MSA,0.0,0.0,NaN,NaN,0,0
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,Rent,Employed,lzgpxyit,"MSA, Principle City",1.0,0.0,fcxhlnwr,cmhcxjea,0,0
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,Own,NaN,lzgpxyit,"MSA, Not Principle City",0.0,0.0,NaN,NaN,0,1


In [12]:
pp.ProfileReport(join_df)

In [306]:
train.dtypes

h1n1_concern                   float64
h1n1_knowledge                 float64
behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
doctor_recc_h1n1               float64
doctor_recc_seasonal           float64
chronic_med_condition          float64
child_under_6_months           float64
health_worker                  float64
health_insurance               float64
opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
age_group                       object
education                       object
race                            object
sex                             object
income_poverty           

In [198]:
num_cols = train.select_dtypes('number').columns

In [199]:
cat_cols = ['race', 'sex', 'marital_status', 'rent_or_own',  'hhs_geo_region','census_msa', 'employment_industry', 'employment_occupation']

In [200]:
ord_cols = ['age_group', 'education',  'income_poverty','employment_status']

In [201]:
for col in (cat_cols+ord_cols):
    train[col] = train[col].fillna(value='None')

In [202]:
for col in num_cols:
    train[col] = train[col].fillna(train[col].mean())

In [203]:
for col in (cat_cols+ord_cols):
    test[col] = test[col].fillna(value='None')

In [204]:
for col in num_cols:
    test[col] = test[col].fillna(test[col].mean())

In [210]:
enc = OrdinalEncoder()

In [212]:
enc.fit(train[ord_cols])

OrdinalEncoder()

In [216]:
train[ord_cols] = enc.transform(train[ord_cols])

C:\Users\grego\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)
C:\Users\grego\anaconda3\envs\learn-env\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


ValueError: Found unknown categories [0.0, 1.0, 2.0, 3.0, 4.0] in column 0 during transform

In [220]:
enc.fit(train[cat_cols])

OrdinalEncoder()

In [221]:
train[cat_cols] = enc.transform(train[cat_cols])

In [222]:
train

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000,...,2.0,2.0,1.0,2.0,8.0,2.0,0.0,0.0,0.0,0.0
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,...,2.0,2.0,2.0,0.0,1.0,0.0,0.0,0.0,13.0,20.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.220312,...,0.0,2.0,1.0,0.0,9.0,0.0,2.0,0.0,15.0,22.0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.000000,0.000000,...,2.0,2.0,2.0,2.0,5.0,1.0,0.0,0.0,0.0,0.0
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.000000,0.000000,...,0.0,0.0,1.0,0.0,9.0,0.0,1.0,0.0,19.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000,...,0.0,2.0,1.0,2.0,9.0,2.0,0.0,0.0,0.0,0.0
26703,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,0.000000,1.000000,...,0.0,2.0,2.0,0.0,6.0,1.0,1.0,0.0,5.0,3.0
26704,2.0,2.0,0.0,1.0,1.0,1.0,1.0,0.0,1.000000,0.000000,...,3.0,2.0,1.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0


In [223]:
scaler = StandardScaler()

In [224]:
scaler.fit(train)

StandardScaler()

In [225]:
normalized_train = scaler.transform(train)

In [231]:
normalized_train_df = pd.DataFrame(normalized_train)

In [240]:
normalized_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       26707 non-null  float64
 1   1       26707 non-null  float64
 2   2       26707 non-null  float64
 3   3       26707 non-null  float64
 4   4       26707 non-null  float64
 5   5       26707 non-null  float64
 6   6       26707 non-null  float64
 7   7       26707 non-null  float64
 8   8       26707 non-null  float64
 9   9       26707 non-null  float64
 10  10      26707 non-null  float64
 11  11      26707 non-null  float64
 12  12      26707 non-null  float64
 13  13      26707 non-null  float64
 14  14      26707 non-null  float64
 15  15      26707 non-null  float64
 16  16      26707 non-null  float64
 17  17      26707 non-null  float64
 18  18      26707 non-null  float64
 19  19      26707 non-null  float64
 20  20      26707 non-null  float64
 21  21      26707 non-null  float64
 22

In [234]:
X_train, X_test, y_train, y_test = train_test_split(normalized_train_df, labels, test_size=0.3, random_state=10)

In [235]:
categorical_features_indices = np.where(X_train.dtypes != np.float)[0]
categorical_features_indices

array([], dtype=int64)

In [236]:
params = {'iterations':500,
         'learning_rate':0.05745075659543725,
         'random_strength':4,
         'bagging_temperature':8,
         'max_bin':5,
         'grow_policy':'Lossguide',
         'min_data_in_leaf':7,
         'max_depth':6,
         'l2_leaf_reg':11.323094517862078,
         'one_hot_max_size':10,
         'auto_class_weights':'Balanced',
         'eval_metric':'AUC',
         'verbose':False,
          }

In [237]:
model_classifier_h1n1 = CatBoostClassifier(**params)

In [238]:
model_classifier_h1n1.fit(X_train, y_train.h1n1_vaccine, eval_set=(X_test, y_test.h1n1_vaccine), use_best_model=True, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [239]:
model_classifier_seasonal.fit(X_train, y_train.seasonal_vaccine)

CatBoostError: Invalid type for cat_feature[non-default value idx=0,feature_idx=21]=-0.8139275173239486 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [182]:
predictions_se = model_classifier_seasonal.predict_proba(X_test)

In [183]:
predictions_se = predictions_se[:,1].reshape(-1,1)

In [168]:
params2 = {'iterations':1500,
         'cat_features': categorical_features_indices,
         'learning_rate':0.015393853845985255,
         'random_strength':3,
         'bagging_temperature':7,
         'max_bin':8,
         'grow_policy':'Lossguide',
         'min_data_in_leaf':1,
         'max_depth':5,
         'l2_leaf_reg':0.03977425239611938,
         'one_hot_max_size':500,
         'auto_class_weights':'Balanced',
         'eval_metric':'AUC',
         'verbose':False,
          }

In [169]:
model_classifier_seasonal = CatBoostClassifier(**params2)

In [170]:
model_classifier_seasonal.fit(X_train, y_train.seasonal_vaccine, eval_set=(X_test, y_test.seasonal_vaccine), use_best_model=True, plot=True);

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [172]:
model_classifier_seasonal.fit(X_train, y_train.seasonal_vaccine)

In [173]:
predictions_se = model_classifier_seasonal.predict_proba(X_test)

In [174]:
predictions_se = predictions_se[:,1].reshape(-1,1)

In [26]:
feat_importance = [t for t in zip(train, model_classifier.get_feature_importance())]
feat_importance_df = pd.DataFrame(data=feat_importance, columns=['Feature', 'VarImp'])
feat_importance_df = feat_importance_df.sort_values('VarImp', ascending=False)
feat_importance_df.head(25)

,Feature,VarImp
19,opinion_seas_risk,14.957622
18,opinion_seas_vacc_effective,14.902562
21,age_group,11.664254
10,doctor_recc_seasonal,10.988042
20,opinion_seas_sick_from_vacc,3.195868
29,hhs_geo_region,3.121720
25,income_poverty,3.026821
15,opinion_h1n1_vacc_effective,2.684029
13,health_worker,2.323966
0,h1n1_concern,2.313667


In [31]:
import pickle
filepath = 'data/models/Catboost/h1n1/model_1.pickl'
pickle.dump(model_classifier_h1n1, open(filepath, 'wb'))

In [32]:
filepath = 'data/models/Catboost/seasonal/model_1.pickl'
pickle.dump(model_classifier_seasonal, open(filepath, 'wb'))

In [127]:
test = pd.read_csv('data/test_set_features.csv', index_col='respondent_id')

In [184]:
model_classifier_seasonal.fit(train, labels.seasonal_vaccine)

In [185]:
final_se = model_classifier_seasonal.predict_proba(test)

In [186]:
final_se = final_se[:,1].reshape(-1,1)

In [187]:
model_classifier_h1n1.fit(train, labels.h1n1_vaccine)

In [188]:
final_h1 = model_classifier_h1n1.predict_proba(test)

In [189]:
final_h1 = final_h1[:,1].reshape(-1,1)

In [190]:
submission_df = pd.read_csv('data/submission_format.csv', index_col='respondent_id')

In [191]:
np.testing.assert_array_equal(test.index.values, submission_df.index.values)

In [192]:
submission_df

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.5,0.7
26708,0.5,0.7
26709,0.5,0.7
26710,0.5,0.7
26711,0.5,0.7
...,...,...
53410,0.5,0.7
53411,0.5,0.7
53412,0.5,0.7


In [195]:
submission_df["h1n1_vaccine"] = final_h1
submission_df["seasonal_vaccine"] = final_se

submission_df.head(50)

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.385558,0.272421
26708,0.086409,0.043102
26709,0.413107,0.729557
26710,0.876236,0.887549
26711,0.649447,0.485935
26712,0.942948,0.979102
26713,0.632904,0.710694
26714,0.455117,0.149961
26715,0.097275,0.126661


In [194]:
date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
submission_df.to_csv(f'./{date}submssion_catboost_optuna.csv', index=True)